In [1]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import dataset, dataloader
import pandas as pd
import nltk
import os
import glovar
import collections
from data import toxic
toxic.prepare()

Preparing necessary preliminaries...
Success.


# Skip-Gram Implementation Practice

Your goal is to implement the Skip-Gram model in PyTorch, including pre-processing. 

Pre-processing is an important step in deep learning with text, and you should learn it now. Even though the Skip-Gram model is different from tasks such as sentence classification, these pre-processing skills are relevant to working with text in general. 

TO SKIP THE PRE-PROCESSING part of the tutorial, skip to number (5) and refer to the instructions. But it is recommended to make sure you are solid with these pre-processing skills.

This tutorial assumes you are familiar with the basics of PyTorch. If not, you should only need review this tutorial first:

https://github.com/jcjohnson/pytorch-examples

Stages in this tutorial:
1. Load the Data
2. Tokenize the Data
3. Build the Vocab Dictionary
4. Prepare Training Pairs
5. Implement Negative Sampling
6. Prepare Data for the Network
7. Code the Model
8. Train the Model
9. Visualize the Embeddings

Tips are provided for the pre-processing stage to make it smoother. Try not to use them. If you have to, put aside some time to learn those skills properly.

You will see validation cells along the way with `assert` calls. Run these to make sure you haven't made any mistakes along the way that will prevent you from proceeding. You will need to do the steps in order.

## 1. Load the Data

We will use the data from Kaggle's Toxic Comment classification task. This data is very noisy, so we have selected a subset of the data that reduces this noise to make this tutorial easier.

The files are provided in the `data` directory. They come in .csv format so we'll use pandas to handle them. If you haven't learned how to use pandas, do it! It is a very useful tool.

The next cell defines the paths to these files.

In [2]:
train_file_path = os.path.join(glovar.DATA_DIR, 'train.csv')
test_file_path = os.path.join(glovar.DATA_DIR, 'test.csv')

### To Do

Load both .csv files into pandas `DataFrame` objects called `train` and `test` and see what they look like by calling the `head()` function on one of them. Identify the name of the column that contains the text of the comments. 

Tips:
- If you don't know pandas, try using `pd.read_csv()`, but put aside some time to learn pandas properly

In [3]:
# train = ???
# test = ???

## 2. Tokenization

We need to determine the set of all tokens in our dataset. We therefore need to separate each comment string into individual tokens, then determine the unique set of those tokens. We focus on the tokenization step first.

We will use `nltk` for tokenization because it is lightweight. The `nltk` package defines a function called `word_tokenize()` that you can use.

### To Do
Given the `DataFrame`s above, obtain a `set` of all unique tokens from <strong>both</strong> the train and test sets. We need to account for all the data in our vocabulary. This variable should be called `token_set`.

Tips:
- The rows of `DataFrame` objects `df1` and `df1` can be joined together using `df1.append(df2)`
- You can get an `array` of values in a `DataFrame` `df` column by calling `df['column_name'].values`

In [4]:
# token_set = ???

In [9]:
assert len(token_set) == len(toxic.get_token_set())

## 3. Build the Vocab Dictionary

We need to associate a unique `int` index with every unique token, and provide a map for lookup. A high-level view of text processing is often: 
1. receive text as input
2. tokenize that text to obtain tokens
3. use those tokens to obtain integer indices
4. use those indices to lookup word vectors
5. use those vectors as input to a neural network.

We focus on (3) now.

### To Do
Use the `token_set` to build a `dict` object called `vocab` that has every unique token in the `token_set` as an index and unique integers as `values`. <strong>Also</strong>, since we will add a padding token for recurrent neural network processing later, make sure your values start from `1` and not `0` so that we can keep `0` for padding later.

Tips:
- The python `zip()` function can be used to bring two lists together - e.g. tokens and indices
- The `dict()` constructor can take a zipped object as input, mapping the first position to index and second to value

In [25]:
# vocab = ???

In [6]:
assert len(vocab) == len(toxic.get_vocab())
assert isinstance(list(vocab.keys())[0], str)
assert isinstance(list(vocab.values())[0], int)
assert 0 not in vocab.values()

We will also define a reverse lookup dictionary (integer indexes as keys and string tokens as values) for later use.

In [26]:
rev_vocab = {v: k for k, v in vocab.items()}

## 4. Prepare the Training Pairs

We need to present two words at a time to the network to train our Skip-Gram: a center word and a context word. We therefore need to determine these pairs beforehand.

Before coding deep learning models it is necessary to first fully think through how we are going to present the data to the network. This will avoid having to make annoying changes that might follow from small details that are easy to overlook.

We know we are going to present two words at a time: a center word, and a context word. But how are we going to present them: as tokens, or as indices? These details matter when you code the forward pass of the network: if you try a word vector lookup on an embedding matrix with a string, you will see an error. We will use integer indices as it will be slightly faster than adding a dictionary lookup as well at training time.

Since finding the context tokens for all words over all instances in the dataset is not a generally useful skill, we do that for you.

In [11]:
m = 5  # our context window size - experiment with this!
contexts = toxic.get_contexts(m)

The `contexts` variable is a dictionary where the keys are the indices of all the tokens in the dataset, and the values are `set`s of token indices that occur in their contexts. We will sample from these during training.

Finally, we need the frequencies of our words for the negative sampling algorithm.

### To Do

Use the training data and `nltk.word_tokenize` to create a `collections.Counter` object that holds each unique token as a key, and the frequency count as a value.

Tips:
- `Counter` has an `update()` function that can accept lists of key values (i.e. tokens) and automatically does the counting for you

In [7]:
# frequencies = ???

In [ ]:
assert len(frequencies) == len(vocab)
assert isinstance(list(frequencies.keys())[0], str)
assert isinstance(list(frequencies.values())[0], int)
rev_vocab = {v: k for k, v in vocab.items()}

The last thing we need to prepare our data for training is define it as a set of pairs of words. Making a complete pass over this set constitutes one epoch of the data.

PyTorch provides a `torch.utils.data.dataset.Dataset` abstraction for a dataset. It is useful to use this abstraction and its companion the `torch.utils.data.dataloader.DataLoader`, which we will implement in (6). The `Dataset` class is abstract and we must be implemented. Actually, we could just use a list - the interface for a `Dataset` is `__len__(self)` and `__getitem__(self, ix)`, so a list will do here. At other times you might consider implementing a `Dataset`, for example if accessing the data dependended on the state of another object, or if you wanted to implement some more complicated training algorithm. We could also implement one that wrapped all the worked we did above - taking .csv files, tokenizing, preparing the vocab dict, etc - and made it conveniently applicable to any text dataset.

In [67]:
pairs = set([])
for center in contexts.keys():
    pairs.update(tuple(zip([center] * len(contexts[center]), list(contexts[center]))))
data = list(pairs)
print('Number of pairs in the dataset: %s' % len(data))

3736855


As you can see, once we multiply our ~80,000 unique tokens by their context sizes we have many training pairs to process. An efficient implementation with negative sampling will be very important.

## 5. Negative Sampling

If you are skipping pre-processing, uncomment the next cell.

In [3]:
vocab = toxic.get_vocab()
frequencies = toxic.get_frequencies()
contexts = toxic.get_contexts(5)
pairs = set([])
for center in contexts.keys():
    pairs.update(tuple(zip([center] * len(contexts[center]), list(contexts[center]))))
pairs = list(pairs)

To perform negative sampling, we need a function that
- Takes a token index as argument
- Returns the number of negative samples we desire
- Randomly chooses those samples according to

$$
P(w_i) = \frac{f(w_i)^{3/4}}{\sum_{j=0}^n (f(w_j)^{3/4})}
$$

We will define this function as a callable class, since it depends on state information (the `vocab`, `frequencies`, and `contexts`).

In [11]:
class NegativeSampler:
    
    def __init__(self, vocab, frequencies, contexts, num_negs):
        """Create a new NegativeSampler.
        
        Args:
          vocab: Dictionary.
          frequencies: List of integers, the frequencies of each word,
            sorted in word index order.
          contexts: Dictionary.
          num_negs: Integer, how many to negatives to sample.
        """
        self.vocab = vocab
        self.n = len(vocab)
        self.contexts = contexts
        self.num_negs = num_negs
        self.distribution = self.p(list(frequencies.values()))
    
    def __call__(self, tok_ix):
        """Get negative samples.
        
        Args:
          tok_ix: Integer, the index of the center word.
        
        Returns:
          List of integers.
        """
        neg_samples = np.random.choice(
            self.n, 
            size=self.num_negs, 
            p=self.distribution)
        # make sure we haven't sampled center word or its context
        invalid = [-1, tok_ix] + list(self.contexts[tok_ix])
        for i, ix in enumerate(neg_samples):
            if ix in invalid:
                new_ix = -1
                while new_ix in invalid:
                    new_ix = np.random.choice(self.n, 
                                              size=1, 
                                              p=self.distribution)[0]
                neg_samples[i] = new_ix
        return [int(s) for s in neg_samples]
    
    def p(self, freqs):
        """Determine the probability distribution for negative sampling.
        
        Args:
          freqs: List of integers.
        
        Returns:
          numpy.ndarray.
        """
        ### Impelement Me ###
        freqs = np.array(freqs)
        return np.power(freqs, 3/4) / np.sum(np.power(freqs, 3/4))

## 6. Prepare Data for the Network

Whatever kind of data we pass our neural network, we often need to take some pre-processing steps at training and prediction time. In our case, we need to perform negative sampling and prepare whatever data structure we decide to pass to our network.

What is in a batch of data?
- center word
- context word
- negative samples

How will we use it?
- embedding lookup for all words
- dot product of center word with context word and negative samples
- softmax over the resulting values

We must define our target vector for the cross-entropy loss calculation. Without negative sampling this is a probability distribution over the entire vocabulary with a `1` at the index of the context word. With negative sampling, we only want to calculate loss and backpropagate gradient for the words in each pass: context, negative samples. We can decide how to arrange this. A consistent convention will be convenient. With five negative samples, we will put the context word in the first position, so all our "targets" will look like:

$$
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0
\end{bmatrix}
$$

We show a row vector here, but in practice this will be a 1D `Tensor`.

It will also be convenient to perform all our dot products in parallel. Efficient neural net implementations avoid for loops wherever possible. We can parallelize our implementation by stacking the output embeddings into a matrix and performing a matrix multiplication with the input vector.

### To Do

Define a `collate(pairs)` function that accepts a `list` of `tuple`s and returns an appropriate data structure, given the considerations just discussed. The tuples will all look like `(center, context)` - i.e. a pair in our `data` variable. The collate function will need to perform negative sampling. It will therefore depend on a NegativeSampler, so we should wrap it and make it a callable.

This might be a challenge and you will need to think forward to how to perform the embedding lookup and matrix multiplication.

Note: normally we would need to collate labels for training, but since we have a conventional target we don't need to here.

In [5]:
class Collate:
    
    def __init__(self, neg_sampler):
        self.sampler = neg_sampler
    
    def __call__(self, pairs):
        ### Implement Me ###
        batch_size = len(pairs)
        centers = [x[0] for x in pairs]
        contexts = [x[1] for x in pairs]
        context_and_negs = []
        for i in range(batch_size):
            neg_samples = self.sampler(centers[i])
            context_and_negs.append([contexts[i]] + list(neg_samples))
        return centers, context_and_negs

Finally, we will want to create a `DataLoader` with our collate function. This will take care of some nice things for us: 
- shuffling our training pairs each epoch
- facilitating enumeration of batches for training
- applying our collate function to each batch
- parallelizing this work on the CPU whilst our GPU processes our model

In [6]:
def get_data_loader(data, batch_size, collate_fn):
    return dataloader.DataLoader(data, 
                                 batch_size=batch_size, 
                                 shuffle=True, 
                                 num_workers=1, 
                                 collate_fn=collate_fn)

## 7. Code the Model

For each training pair, we need to:
1. Perform embedding lookup for the center word (from $\mathbf{V}$)
2. Lookup the context and negative sample embeddings (from $\mathbf{U}$)
3. Perform a dot product of the center word embeddings with context and negative sample embeddings
4. Pass the results of the dot products to the softmax function
5. Compute the loss given the labels
6. Use the loss to update the vectors

Below is a template for the model. We will walk through the steps here. Look for `### Implement Me ###` in the model template for the parts you need to complete.

### To Do

1. Define the network parameters in the `__init__` method. Specifically we need $\mathbf{V} \in \Bbb{R}^{\text{vocab size} \times \text{emb_dim}}$ and $\mathbf{U} \in \Bbb{R}^{\text{emb_dim} \times \text{vocab size}}$. Define them as `nn.Parameter` objects so that PyTorch will automatically add them to `self.parameters()` on the `nn.Module` to make it easier to work with.
2. 

In [37]:
class SkipGram(nn.Module):
    """SkipGram Model."""
    
    def __init__(self, vocab, emb_dim, num_negs, lr):
        """Create a new SkipGram.
        
        Args:
          vocab: Dictionary, our vocab dict with token keys and index values.
          emb_dim: Integer, the size of word embeddings.
          num_negs: Integer, the number of non-context words to sample.
          lr: Float, the learning rate for gradient descent.
        """
        super(SkipGram, self).__init__()
        self.vocab = vocab
        self.n = len(vocab)  # size of the vocab
        self.emb_dim = emb_dim
        self.num_negs = num_negs
        
        ### Implement Me: define V and U ###
        
        ### Implement Me: initialize V and U with unform distribution in [-0.01, 0.01] ###
        
        self.V = nn.Embedding(self.n, emb_dim)
        self.U = nn.Embedding(self.n, emb_dim)
        #self.V = nn.Parameter(torch.Tensor(self.n, emb_dim), requires_grad=True)
        #self.U = nn.Parameter(torch.Tensor(emb_dim, self.n), requires_grad=True)
        nn.init.uniform(self.V.weight, a=-0.01, b=0.01)
        nn.init.uniform(self.U.weight, a=-0.01, b=0.01)        
        
        # Adam is a good optimizer and will converge faster than SGD
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        
        if torch.cuda.is_available():
            self.cuda()
    
    def forward(self, batch):
        """Compute the forward pass of the network.
        
        1. Lookup embeddings for center and context words
            - use self.lookup()
        2. Sample negative samples
            - use self.negative_samples()
        2. Calculate the probability estimates
            - make sure to implement and use self.softmax()
        3. Calculate the loss
            - using self.loss()
        
        Args:
          batch: whatever data structure you decided to make for a batch.
        
        Returns:
          loss (torch.autograd.Variable).
        """
        ### Implement Me: lookup center words ###
        ### Implement Me: lookup context words ###
        ### Implement Me: lookup negative words ###
        cents, conts_negs = batch
        cents = self.V(self.lookup_tensor(cents))
        conts_negs = self.U(self.lookup_tensor(conts_negs)).permute([0, 2, 1])
        logits = cents.matmul(conts_negs)
        preds = self.softmax(logits)
        targets = self.targets(cents.shape[0])
        return self.loss(preds, targets)
    
    def lookup_tensor(self, indices):
        """Lookup embeddings given indices.
        
        Args:
          embedding: nn.Parameter, an embedding matrix.
          indices: List of integers, the indices to lookup.
        
        Returns:
          torch.autograd.Variable of shape [len(indices), emb_dim]. A matrix 
            with horizontally stacked word vectors.
        """
        if torch.cuda.is_available():
            return Variable(torch.LongTensor(indices),
                            requires_grad=False).cuda()
        else:
            return Variable(torch.LongTensor(indices),
                            requires_grad=False)
    
    def loss(self, preds, targets):
        """Compute cross-entropy loss.
        
        Implement this for practice, don't use the built-in PyTorch function.
        
        Args:
          preds: Tensor of shape [batch_size, vocab_size], our predictions.
          targets: List of integers, the vocab indices of target context words.
        """
        ### Implement Me ###
        return -1 * torch.sum(targets * torch.log(preds))
    
    def optimize(self, loss):
        """Optimization step.
        
        Args:
          loss: Scalar.
        """
        # Remove any previous gradient from our tensors before calculating again.
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()        
    
    def softmax(self, logits):
        """Compute the softmax function.
        
        Implement this for practice, don't use the built-in PyTorch function.
        
        Args:
          logits: Tensor of shape [batch_size, vocab_size].
        
        Returns:
          Tensor of shape [batch_size, vocab_size], our predictions.
        """
        ### Impelement Me ###
        return torch.exp(logits) / torch.sum(torch.exp(logits))
    
    def targets(self, batch_size):
        """Get the conventional targets for the batch.
        
        Args:
          batch_size: Integer.
        
        Returns:
          torch.LongTensor.
        """
        targets = torch.zeros((batch_size, self.num_negs + 1))
        targets[:, 0] = 1
        return Variable(targets, requires_grad=False)

## 8. Train the Model

Training algorithms should consider
- Learning rate annealing
- Convergence conditions
- Early stopping

Here we will not anneal the learning rate for simplicity and pre-define a 10 epoch limit. Training will stop early if loss doesn't improve after one epoch.

You should experiment with different training strategies in your own work.

In [39]:
# Hyperparameters
epoch = 0
max_epochs = 10
emb_dim = 50
num_negs = 5
lr = 0.02
batch_size = 16

sampler = NegativeSampler(vocab, frequencies, contexts, num_negs)
collate = Collate(sampler)
data_loader = get_data_loader(pairs, batch_size, collate)
model = SkipGram(vocab, emb_dim, num_negs, lr)
if torch.cuda.is_available():
    model.cuda()

while epoch <= max_epochs:
    epoch += 1
    print('Epoch %s' % epoch)
    for step, batch in enumerate(data_loader):
        loss = model.forward(batch)
        model.optimize(loss)
        if step % 100 == 0:
            print('Step %s\t\tLoss %s' % (step, loss.data.cpu().numpy()[0]))

Epoch 1
Step 0		Loss 1878.2468
Step 100		Loss 1881.4657
Step 200		Loss 1879.0082
Step 300		Loss 1881.7604
Step 400		Loss 1892.5626
Step 500		Loss 1909.8878


Process Process-17:
  File "/home/hanshan/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/hanshan/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hanshan/anaconda3/envs/py35/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 42, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-5-0aced094a953>", line 13, in __call__
    neg_samples = self.sampler(centers[i])
KeyboardInterrupt
  File "<ipython-input-11-307ab89c14cd>", line 35, in __call__
    if ix in invalid:


KeyboardInterrupt: 